# Prototype of the Averaging Function to Support Incremental Updates to Data Sources

## Imports, constants, and data load

In [2]:
import pandas as pd
import numpy as np
import pathlib
import os
import sys

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

from data_pipeline.etl.base import ExtractTransformLoad
from data_pipeline.score import field_names

DATA_DIR = pathlib.Path.cwd().parent / "data"

In [14]:
path_to_score_census_acs_file = DATA_DIR / "dataset/census_acs/usa.csv"
path_to_xwalk = DATA_DIR / "national_2020_census_tract_to_2010_census_tract_relationship_file.txt"

census_acs = pd.read_csv(
    path_to_score_census_acs_file,
    dtype={'GEOID10_TRACT': "string"},
)

xwalk = pd.read_csv(
    path_to_xwalk,
    sep = '|',
    dtype={'GEOID_TRACT_10': "string",
           'GEOID_TRACT_20': "string"},
).rename(columns={'GEOID_TRACT_10': 'GEOID10_TRACT',
                  'GEOID_TRACT_20': 'GEOID20_TRACT'}
        )

In [15]:
xwalk = xwalk[['GEOID10_TRACT', 'GEOID20_TRACT']]

In [18]:
merged_df = pd.merge(census_acs, xwalk, how='left', on='GEOID10_TRACT')

In [23]:
missing_tract_20s_merged = set(xwalk.GEOID20_TRACT) - set(merged_df.GEOID20_TRACT) 
print(len(missing_tract_20s_merged))
missing_tract_20s_merged

155


{'02158000100',
 '04019002704',
 '04019002906',
 '04019004131',
 '04019004133',
 '04019980001',
 '04019980002',
 '04019980100',
 '36053030101',
 '36053030102',
 '36053030103',
 '36053030200',
 '36053030300',
 '36053030401',
 '36053030402',
 '36053030403',
 '36053030404',
 '36053030601',
 '36053030602',
 '36065024700',
 '36065024800',
 '46102941200',
 '60010950100',
 '60010950200',
 '60010950300',
 '60010950500',
 '60010950600',
 '60010950700',
 '60010950900',
 '60020951800',
 '60030951900',
 '60040952000',
 '60050951000',
 '60050951100',
 '60050951201',
 '60050951202',
 '60050951203',
 '60050951300',
 '60050951500',
 '60050951600',
 '66010950100',
 '66010950200',
 '66010950300',
 '66010950401',
 '66010950402',
 '66010950501',
 '66010950502',
 '66010950701',
 '66010950702',
 '66010950801',
 '66010950802',
 '66010950900',
 '66010951000',
 '66010951100',
 '66010951600',
 '66010951700',
 '66010951800',
 '66010951901',
 '66010951902',
 '66010952200',
 '66010952300',
 '66010952400',
 '660109

In [25]:
missing_tract_10s_merged = set(xwalk.GEOID10_TRACT) - set(merged_df.GEOID10_TRACT) 
print(len(missing_tract_10s_merged))
missing_tract_10s_merged

158


{'02270000100',
 '04019002701',
 '04019002903',
 '04019410501',
 '04019410502',
 '04019410503',
 '04019470400',
 '04019470500',
 '06037930401',
 '36053940101',
 '36053940102',
 '36053940103',
 '36053940200',
 '36053940300',
 '36053940401',
 '36053940403',
 '36053940600',
 '36053940700',
 '36065940000',
 '36065940100',
 '36065940200',
 '36085008900',
 '46113940500',
 '46113940800',
 '46113940900',
 '51515050100',
 '60010950100',
 '60010950200',
 '60010950300',
 '60010950500',
 '60010950600',
 '60010950700',
 '60010950900',
 '60020951800',
 '60030951900',
 '60040952000',
 '60050951000',
 '60050951100',
 '60050951201',
 '60050951202',
 '60050951203',
 '60050951300',
 '60050951500',
 '60050951600',
 '66010950100',
 '66010950200',
 '66010950300',
 '66010950401',
 '66010950402',
 '66010950501',
 '66010950502',
 '66010950701',
 '66010950702',
 '66010950801',
 '66010950802',
 '66010950900',
 '66010951000',
 '66010951100',
 '66010951600',
 '66010951700',
 '66010951800',
 '66010951901',
 '660109

In [26]:
missing_tract_10s_current = set(xwalk.GEOID10_TRACT) - set(census_acs.GEOID10_TRACT) 
print(len(missing_tract_10s_current))
missing_tract_10s_current

158


{'02270000100',
 '04019002701',
 '04019002903',
 '04019410501',
 '04019410502',
 '04019410503',
 '04019470400',
 '04019470500',
 '06037930401',
 '36053940101',
 '36053940102',
 '36053940103',
 '36053940200',
 '36053940300',
 '36053940401',
 '36053940403',
 '36053940600',
 '36053940700',
 '36065940000',
 '36065940100',
 '36065940200',
 '36085008900',
 '46113940500',
 '46113940800',
 '46113940900',
 '51515050100',
 '60010950100',
 '60010950200',
 '60010950300',
 '60010950500',
 '60010950600',
 '60010950700',
 '60010950900',
 '60020951800',
 '60030951900',
 '60040952000',
 '60050951000',
 '60050951100',
 '60050951201',
 '60050951202',
 '60050951203',
 '60050951300',
 '60050951500',
 '60050951600',
 '66010950100',
 '66010950200',
 '66010950300',
 '66010950401',
 '66010950402',
 '66010950501',
 '66010950502',
 '66010950701',
 '66010950702',
 '66010950801',
 '66010950802',
 '66010950900',
 '66010951000',
 '66010951100',
 '66010951600',
 '66010951700',
 '66010951800',
 '66010951901',
 '660109

In [29]:
averaged = merged_df.groupby("GEOID20_TRACT").mean()

In [32]:
[x for x in averaged.columns if 'GEOID' in x]

[]

In [34]:
set(census_acs.columns) - set(averaged.columns)

{'GEOID10_TRACT'}

In [35]:
set(averaged.columns) - set(census_acs.columns)

set()

In [28]:
len(merged_df.GEOID20_TRACT.unique())

85373

In [43]:
merged_df[merged_df.GEOID20_TRACT.str[:2].isin(["60", "66", "69", "78"])]

,GEOID10_TRACT,Total population,Unemployment (percent),Linguistic isolation (percent),Median household income in the past 12 months,Percent of individuals < 100% Federal Poverty Line,Percent of individuals < 150% Federal Poverty Line,"Percent of individuals < 200% Federal Poverty Line, imputed",Total population of individuals < 200% Federal Poverty Line,"Total population of individuals < 200% Federal Poverty Line, imputed",...,Percent two or more races,Percent White,Percent Hispanic or Latino,Percent other races,Percent age under 10,Percent age 10 to 64,Percent age over 64,Percent of individuals below 200% Federal Poverty Line,"Percent of individuals below 200% Federal Poverty Line, imputed and adjusted",GEOID20_TRACT


In [44]:
len(merged_df.GEOID20_TRACT.str[:2].unique())

52

In [50]:
averaged.head()

,Total population,Unemployment (percent),Linguistic isolation (percent),Median household income in the past 12 months,Percent of individuals < 100% Federal Poverty Line,Percent of individuals < 150% Federal Poverty Line,"Percent of individuals < 200% Federal Poverty Line, imputed",Total population of individuals < 200% Federal Poverty Line,"Total population of individuals < 200% Federal Poverty Line, imputed",Median value ($) of owner-occupied housing units,...,Percent Native Hawaiian or Pacific,Percent two or more races,Percent White,Percent Hispanic or Latino,Percent other races,Percent age under 10,Percent age 10 to 64,Percent age over 64,Percent of individuals below 200% Federal Poverty Line,"Percent of individuals below 200% Federal Poverty Line, imputed and adjusted"
GEOID20_TRACT,,,,,,,,,,,,,,,,,,,,,
01001020100,1976.0,0.036111,0.000000,52083.0,0.178962,0.260895,0.355445,646.5,646.5,113300.0,...,0.005360,0.052928,0.607631,0.014180,0.001531,0.112448,0.748834,0.138718,0.355445,0.368791
01001020200,1959.0,0.043651,0.000000,43958.0,0.191340,0.284962,0.435927,745.0,745.0,90500.0,...,0.010720,0.028586,0.382848,0.015314,0.003063,0.124043,0.730985,0.144972,0.435927,0.446561
01001020300,3507.0,0.035196,0.000000,55345.0,0.172512,0.231252,0.293128,1028.0,1028.0,122600.0,...,0.001426,0.027089,0.651554,0.066439,0.066439,0.122327,0.717422,0.160251,0.293128,0.299708
01001020400,3878.0,0.031987,0.023881,59663.0,0.046416,0.119134,0.216864,841.0,841.0,152700.0,...,0.000000,0.031975,0.896854,0.010830,0.000000,0.101341,0.653945,0.244714,0.216864,0.226562
01001020501,10596.0,0.031174,0.019630,66108.0,0.151626,0.181836,0.203222,2119.0,2119.0,186900.0,...,0.000000,0.023688,0.709041,0.044262,0.004624,0.118535,0.743394,0.138071,0.203222,0.190738


In [51]:
census_acs.head()

,GEOID10_TRACT,Total population,Unemployment (percent),Linguistic isolation (percent),Median household income in the past 12 months,Percent of individuals < 100% Federal Poverty Line,Percent of individuals < 150% Federal Poverty Line,"Percent of individuals < 200% Federal Poverty Line, imputed",Total population of individuals < 200% Federal Poverty Line,"Total population of individuals < 200% Federal Poverty Line, imputed",...,Percent Native Hawaiian or Pacific,Percent two or more races,Percent White,Percent Hispanic or Latino,Percent other races,Percent age under 10,Percent age 10 to 64,Percent age over 64,Percent of individuals below 200% Federal Poverty Line,"Percent of individuals below 200% Federal Poverty Line, imputed and adjusted"
0,01073001100,4781,0.009207,0.000000,37030.0,0.150376,0.318797,0.374436,1743,1743.0,...,0.0,0.000000,0.016105,0.003556,0.000000,0.134491,0.661577,0.203932,0.374436,0.378549
1,01073001400,1946,0.115012,0.024510,36066.0,0.281603,0.367934,0.483556,941,941.0,...,0.0,0.003597,0.000000,0.068345,0.077595,0.085303,0.725591,0.189106,0.483556,0.475098
2,01073002000,4080,0.137549,0.019732,27159.0,0.513352,0.616172,0.727976,2917,2917.0,...,0.0,0.009314,0.277451,0.075000,0.020098,0.158333,0.719608,0.122059,0.727976,0.714270
3,01073003802,5291,0.103332,0.000000,38721.0,0.196560,0.340389,0.529012,2799,2799.0,...,0.0,0.008505,0.016065,0.000000,0.000000,0.146664,0.720658,0.132678,0.529012,0.536852
4,01073004000,2533,0.211483,0.044787,18525.0,0.405843,0.585077,0.707856,1793,1793.0,...,0.0,0.045006,0.065930,0.068298,0.000000,0.063561,0.688117,0.248322,0.707856,0.716085


In [53]:
averaged.reset_index()

,GEOID20_TRACT,Total population,Unemployment (percent),Linguistic isolation (percent),Median household income in the past 12 months,Percent of individuals < 100% Federal Poverty Line,Percent of individuals < 150% Federal Poverty Line,"Percent of individuals < 200% Federal Poverty Line, imputed",Total population of individuals < 200% Federal Poverty Line,"Total population of individuals < 200% Federal Poverty Line, imputed",...,Percent Native Hawaiian or Pacific,Percent two or more races,Percent White,Percent Hispanic or Latino,Percent other races,Percent age under 10,Percent age 10 to 64,Percent age over 64,Percent of individuals below 200% Federal Poverty Line,"Percent of individuals below 200% Federal Poverty Line, imputed and adjusted"
0,01001020100,1976.0,0.036111,0.000000,52083.0,0.178962,0.260895,0.355445,646.5,646.5,...,0.005360,0.052928,0.607631,0.014180,0.001531,0.112448,0.748834,0.138718,0.355445,0.368791
1,01001020200,1959.0,0.043651,0.000000,43958.0,0.191340,0.284962,0.435927,745.0,745.0,...,0.010720,0.028586,0.382848,0.015314,0.003063,0.124043,0.730985,0.144972,0.435927,0.446561
2,01001020300,3507.0,0.035196,0.000000,55345.0,0.172512,0.231252,0.293128,1028.0,1028.0,...,0.001426,0.027089,0.651554,0.066439,0.066439,0.122327,0.717422,0.160251,0.293128,0.299708
3,01001020400,3878.0,0.031987,0.023881,59663.0,0.046416,0.119134,0.216864,841.0,841.0,...,0.000000,0.031975,0.896854,0.010830,0.000000,0.101341,0.653945,0.244714,0.216864,0.226562
4,01001020501,10596.0,0.031174,0.019630,66108.0,0.151626,0.181836,0.203222,2119.0,2119.0,...,0.000000,0.023688,0.709041,0.044262,0.004624,0.118535,0.743394,0.138071,0.203222,0.190738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85368,72153750501,6113.0,0.117051,0.782748,20221.0,0.390976,0.643806,0.821821,5009.0,5009.0,...,0.000000,0.011287,0.001472,0.998528,0.149190,0.152462,0.715034,0.132504,0.821821,0.854295
85369,72153750502,2388.0,0.197368,0.635171,18871.0,0.388191,0.531826,0.656198,1567.0,1567.0,...,0.000000,0.012144,0.007956,0.992044,0.156616,0.154104,0.698492,0.147404,0.656198,0.689596
85370,72153750503,2126.0,0.273616,0.680067,13699.0,0.567733,0.668391,0.808090,1718.0,1718.0,...,0.000000,0.015992,0.001881,0.995296,0.223424,0.151458,0.638288,0.210254,0.808090,0.815619
85371,72153750601,4397.0,0.204683,0.583131,23722.0,0.437798,0.567205,0.763475,3357.0,3357.0,...,0.000000,0.006595,0.000000,0.992040,0.181487,0.148055,0.678872,0.173073,0.763475,0.765468
